In [1]:
!pip install torch psutil tqdm numpy rogue-score


ERROR: Could not find a version that satisfies the requirement rogue-score (from versions: none)
ERROR: No matching distribution found for rogue-score

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Config, GPT2Model, DistilBertModel, GPT2TokenizerFast, DistilBertTokenizerFast,GPT2Tokenizer,GPT2LMHeadModel
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import re
import gc  # Add this import at the top
import psutil
from tqdm import tqdm

import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Config, GPT2Model, DistilBertModel, GPT2TokenizerFast, DistilBertTokenizerFast
from typing import Optional, Dict  # Add this import for type hints

def log_to_file(message):
    """Helper function for logging"""
    print(message)

class CrossAttentionFusionLayer(nn.Module):
    def __init__(self, gpt2_dim: int, bert_dim: int, output_dim: int, dropout_rate: float, num_heads: int = 8):
        super().__init__()
        self.gpt2_proj = nn.Linear(gpt2_dim, output_dim)
        self.bert_proj = nn.Linear(bert_dim, output_dim)
        self.cross_attention = nn.MultiheadAttention(embed_dim=output_dim, num_heads=num_heads, dropout=dropout_rate, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.layer_norm = nn.LayerNorm(output_dim)

    def forward(self, gpt2_features: torch.Tensor, bert_features: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
        gpt2_proj = self.gpt2_proj(gpt2_features)  # [batch_size, seq_len, output_dim]
        bert_proj = self.bert_proj(bert_features)  # [batch_size, seq_len, output_dim]
        attn_mask = attention_mask.float().masked_fill(attention_mask == 0, float('-inf')).masked_fill(attention_mask == 1, 0)
        fused_features, _ = self.cross_attention(
            query=gpt2_proj,
            key=bert_proj,
            value=bert_proj,
            key_padding_mask=attention_mask == 0
        )
        fused_features = self.dropout(fused_features) + gpt2_proj  # Residual connection
        return self.layer_norm(fused_features)

class HybridGPT2DistilBERTMultiTask(nn.Module):
    def __init__(self, num_intents: int, num_categories: int, num_ner_labels: int,
                 dropout_rate: float, loss_weights: Optional[Dict[str, float]] = None,
                 ner_class_weights: Optional[torch.Tensor] = None,
                 category_class_weights: Optional[torch.Tensor] = None,
                 intent_class_weights: Optional[torch.Tensor] = None):
        super().__init__()
        log_to_file("Initializing model...")
        self.gpt2_config = GPT2Config.from_pretrained('gpt2')
        self.gpt2 = GPT2Model.from_pretrained('gpt2')
        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')

        # Freeze all layers
        for param in self.gpt2.parameters():
            param.requires_grad = False
        for param in self.distilbert.parameters():
            param.requires_grad = False
        log_to_file("All GPT-2 and DistilBERT layers remain frozen")

        gpt2_dim = self.gpt2_config.n_embd
        bert_dim = self.distilbert.config.hidden_size
        hidden_size = gpt2_dim  # Keeping output dim same as GPT-2 for consistency

        self.fusion_layer = CrossAttentionFusionLayer(gpt2_dim, bert_dim, hidden_size, dropout_rate)

        self.intent_head = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, num_intents)
        )
        self.category_head = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, num_categories)
        )
        self.ner_head = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, num_ner_labels)
        )

        # Set default loss weights if not provided
        self.loss_weights = loss_weights or {
            'intent': 0.3,
            'category': 0.3,
            'ner': 0.4
        }

        # Loss functions with class weights
        self.intent_loss_fn = nn.CrossEntropyLoss(weight=intent_class_weights) if intent_class_weights is not None else nn.CrossEntropyLoss()
        self.category_loss_fn = nn.CrossEntropyLoss(weight=category_class_weights) if category_class_weights is not None else nn.CrossEntropyLoss()
        self.ner_loss_fn = nn.CrossEntropyLoss(weight=ner_class_weights) if ner_class_weights is not None else nn.CrossEntropyLoss()

    def forward(self, gpt2_input_ids: torch.Tensor, gpt2_attention_mask: torch.Tensor,
                distilbert_input_ids: torch.Tensor, distilbert_attention_mask: torch.Tensor,
                intent_labels: Optional[torch.Tensor] = None,
                category_labels: Optional[torch.Tensor] = None,
                ner_labels: Optional[torch.Tensor] = None) -> Dict[str, torch.Tensor]:
        gpt2_outputs = self.gpt2(input_ids=gpt2_input_ids, attention_mask=gpt2_attention_mask)
        distilbert_outputs = self.distilbert(input_ids=distilbert_input_ids, attention_mask=distilbert_attention_mask)

        gpt2_features = gpt2_outputs.last_hidden_state
        bert_features = distilbert_outputs.last_hidden_state

        fused_features = self.fusion_layer(gpt2_features, bert_features, gpt2_attention_mask)

        batch_size = fused_features.shape[0]
        sequence_lengths = gpt2_attention_mask.sum(dim=1) - 1
        last_token_indexes = sequence_lengths.unsqueeze(-1).unsqueeze(-1).repeat(1, 1, fused_features.shape[-1])
        sequence_repr = torch.gather(fused_features, 1, last_token_indexes).squeeze(1)

        intent_logits = self.intent_head(sequence_repr)
        category_logits = self.category_head(sequence_repr)
        ner_logits = self.ner_head(fused_features)

        output_dict = {
            'intent_logits': intent_logits,
            'category_logits': category_logits,
            'ner_logits': ner_logits
        }

        if all(label is not None for label in [intent_labels, category_labels, ner_labels]):
            intent_loss = self.intent_loss_fn(intent_logits, intent_labels)
            category_loss = self.category_loss_fn(category_logits, category_labels)
            active_loss = gpt2_attention_mask.view(-1) == 1
            active_logits = ner_logits.view(-1, ner_logits.size(-1))[active_loss]
            active_labels = ner_labels.view(-1)[active_loss]
            ner_loss = self.ner_loss_fn(active_logits, active_labels)

            total_loss = (self.loss_weights['intent'] * intent_loss +
                          self.loss_weights['category'] * category_loss +
                          self.loss_weights['ner'] * ner_loss)

            output_dict.update({
                'loss': total_loss,
                'intent_loss': intent_loss,
                'category_loss': category_loss,
                'ner_loss': ner_loss
            })

        return output_dict

def inference_hybrid(model, text, gpt2_tokenizer, distilbert_tokenizer, label_encoders, max_length, device):
    """Run inference with the HybridGPT2DistilBERTMultiTask model with confidence scores as decimals"""
    model.eval()
    
    # Tokenize input
    gpt2_encoding = gpt2_tokenizer(
        text,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    distilbert_encoding = distilbert_tokenizer(
        text,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    inputs = {
        "gpt2_input_ids": gpt2_encoding["input_ids"].to(device),
        "gpt2_attention_mask": gpt2_encoding["attention_mask"].to(device),
        "distilbert_input_ids": distilbert_encoding["input_ids"].to(device),
        "distilbert_attention_mask": distilbert_encoding["attention_mask"].to(device)
    }

    # Run model
    with torch.no_grad():
        outputs = model(**inputs)

    # Get softmax probabilities for confidence scores
    intent_logits = outputs["intent_logits"]
    category_logits = outputs["category_logits"]
    ner_logits = outputs["ner_logits"]

    # Apply softmax to get probabilities
    intent_probs = torch.nn.functional.softmax(intent_logits, dim=-1)[0]
    category_probs = torch.nn.functional.softmax(category_logits, dim=-1)[0]
    ner_probs = torch.nn.functional.softmax(ner_logits, dim=-1)

    # Get top predictions and their confidences
    intent_pred = torch.argmax(intent_probs).cpu().item()
    intent_confidence = intent_probs[intent_pred].cpu().item()  # As decimal (0.0-1.0)

    category_pred = torch.argmax(category_probs).cpu().item()
    category_confidence = category_probs[category_pred].cpu().item()  # As decimal (0.0-1.0)

    ner_preds = torch.argmax(ner_probs, dim=-1).cpu().numpy()[0]
    ner_confidences = torch.max(ner_probs, dim=-1)[0][0].cpu().numpy()  # Get max probability for each token

    # Map to labels
    intent_decoder = {v: k for k, v in label_encoders["intent_encoder"].items()}
    category_decoder = {v: k for k, v in label_encoders["category_encoder"].items()}
    ner_decoder = {v: k for k, v in label_encoders["ner_label_encoder"].items()}

    intent_label = intent_decoder[intent_pred]
    category_label = category_decoder[category_pred]

    tokens = gpt2_tokenizer.convert_ids_to_tokens(inputs["gpt2_input_ids"][0].tolist())
    seq_len = int(inputs["gpt2_attention_mask"][0].sum().item())
    ner_labels = [ner_decoder[pred] for pred in ner_preds[:seq_len]]

    # Extract entities from NER labels with confidence
    entities = []
    current_entity = None
    entity_tokens = []
    entity_confidences = []
    entity_type = None

    for i, (token, label, confidence) in enumerate(zip(tokens[:seq_len], ner_labels, ner_confidences[:seq_len])):
        if label.startswith("B-"):
            # If we were tracking an entity, save it before starting a new one
            if current_entity is not None:
                entity_text = gpt2_tokenizer.convert_tokens_to_string(entity_tokens).strip()
                if entity_text:
                    # Calculate average confidence for the entity (as decimal)
                    avg_confidence = sum(entity_confidences) / len(entity_confidences)
                    entities.append({
                        "entity": entity_text,
                        "label": entity_type,
                        "confidence": avg_confidence
                    })

            # Start a new entity
            current_entity = label[2:]
            entity_type = label[2:]
            entity_tokens = [token]
            entity_confidences = [confidence]

        elif label.startswith("I-") and current_entity == label[2:]:
            # Continue current entity
            entity_tokens.append(token)
            entity_confidences.append(confidence)

        elif current_entity is not None:
            # End of an entity
            entity_text = gpt2_tokenizer.convert_tokens_to_string(entity_tokens).strip()
            if entity_text:
                # Calculate average confidence for the entity (as decimal)
                avg_confidence = sum(entity_confidences) / len(entity_confidences)
                entities.append({
                    "entity": entity_text,
                    "label": entity_type,
                    "confidence": avg_confidence
                })
            current_entity = None
            entity_tokens = []
            entity_confidences = []
            entity_type = None

    # Check for unfinished entity
    if current_entity is not None:
        entity_text = gpt2_tokenizer.convert_tokens_to_string(entity_tokens).strip()
        if entity_text:
            # Calculate average confidence for the entity (as decimal)
            avg_confidence = sum(entity_confidences) / len(entity_confidences)
            entities.append({
                "entity": entity_text,
                "label": entity_type,
                "confidence": avg_confidence
            })

    return {
        "intent": {"label": intent_label, "confidence": intent_confidence},
        "category": {"label": category_label, "confidence": category_confidence},
        "ner": entities
    }

def generate_response(model, tokenizer, instruction, classification, max_length=512, device="cuda"):
    """Generate a response using GPT-2 model"""
    model.eval()

    # Extract classification details
    intent = classification["intent"]["label"] if isinstance(classification["intent"], dict) else classification["intent"]
    category = classification["category"]["label"] if isinstance(classification["category"], dict) else classification["category"]

    # Clean up intent and category strings
    if isinstance(intent, str) and "[" in intent:
        intent = intent.strip("[]'")
    if isinstance(category, str) and "[" in category:
        category = category.strip("[]'")

    # Format entity information
    entities_text = ""
    if "ner" in classification and classification["ner"]:
        entities = []
        for entity in classification["ner"]:
            if isinstance(entity, dict) and "entity" in entity and "label" in entity:
                entities.append(f"{entity['entity']} ({entity['label']})")
        entities_text = ", ".join(entities)
    else:
        entities_text = "none"

    # Create a prompt for generating response
    input_text = f"[INST] User query: {instruction}\n\n" \
                 f"Based on the following classification:\n" \
                 f"- Intent: {intent}\n" \
                 f"- Category: {category}\n" \
                 f"- Entities: {entities_text}\n\n" \
                 f"Provide a helpful customer service response: [RESP]"

    # Tokenize input
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    attention_mask = torch.ones_like(input_ids).to(device)

    try:
        # Generate response
        with torch.no_grad():
            output_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                num_beams=5,
                no_repeat_ngram_size=2,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode the generated text
        generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=False)

        # Extract response part
        if "[RESP]" in generated_text:
            response = generated_text.split("[RESP]")[1].strip()
            # Clean up any trailing tokens
            if "[EOS]" in response:
                response = response.split("[EOS]")[0].strip()
        else:
            # Fallback extraction
            response = generated_text[len(input_text):].strip()
        # Format steps if present
        steps_pattern = re.search(r'(\d+)\.\s+([A-Z])', response)
        if steps_pattern or "step" in response.lower() or "follow" in response.lower():
            # Format steps to be on separate lines
            for i in range(1, 10):
                step_marker = f"{i}. "
                if step_marker in response and f"\n{i}. " not in response:
                    response = response.replace(step_marker, f"\n{i}. ")

            # Clean up any excess newlines
            response = re.sub(r'\n\s*\n', '\n\n', response)
            response = response.lstrip('\n')

        # Clean any technical artifacts
        response = re.sub(r'https?://\S+', '', response)  # Remove URLs
        response = re.sub(r'<[^>]*>', '', response)  # Remove HTML tags
        response = re.sub(r'\{\s*"[^"]*":', '', response)  # Remove JSON-like content
        response = re.sub(r'\s+', ' ', response).strip()  # Clean up whitespace

        return response

    except Exception as e:
        print(f"Error in generate_response: {e}")
        return f"I apologize, but I couldn't generate a response. Error: {str(e)}"

# Memory measurement functions
def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / 1024 / 1024  # Convert bytes to MB

# Fix the get_peak_memory_usage function
def get_peak_memory_usage(func, *args, **kwargs):
    device_param = kwargs.pop('device', None)  # Remove device parameter before calling func
    
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()
    
    start_mem = get_memory_usage()
    result = func(*args, **kwargs)  # Call without device parameter
    end_mem = get_memory_usage()
    
    peak_gpu_mem = 0
    if torch.cuda.is_available() and device_param == "cuda":
        peak_gpu_mem = torch.cuda.max_memory_allocated() / 1024 / 1024  # MB
        return result, peak_gpu_mem
    else:
        return result, max(0, end_mem - start_mem)

class PerformanceTest:
    def __init__(self, model_paths, test_data_path, num_samples=50):
        self.output_dir = model_paths["hybrid_model_dir"]
        self.generation_model_path = model_paths["generation_model_path"]
        self.generation_tokenizer_path = model_paths["generation_tokenizer_path"]
        self.test_data_path = test_data_path
        self.num_samples = num_samples
        self.devices = ["cpu", "cuda"] if torch.cuda.is_available() else ["cpu"]
        self.results = {device: [] for device in self.devices}
        self.summary = {device: {"classification_time": [], "generation_time": [], "overall_time": [], "memory_usage": []} for device in self.devices}
    
    def load_models(self, device):
        print(f"\nLoading models on {device.upper()}...")
        
        # Load hybrid model for classification
        encoders_path = os.path.join(self.output_dir, "label_encoders.json")
        hyperparams_path = os.path.join(self.output_dir, "hyperparameters.json")
        model_path = os.path.join(self.output_dir, "hybrid_model.pth")
        
        print(f"Loading from: {encoders_path}")
        with open(encoders_path, 'r', encoding='utf-8') as f:
            self.label_encoders = json.load(f)
            
        with open(hyperparams_path, 'r', encoding='utf-8') as f:
            self.hyperparameters = json.load(f)
                
        self.gpt2_tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
        self.distilbert_tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
        
        if self.gpt2_tokenizer.pad_token is None:
            self.gpt2_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        if self.distilbert_tokenizer.pad_token is None:
            self.distilbert_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
                
        self.classification_model = HybridGPT2DistilBERTMultiTask(
            num_intents=len(self.label_encoders["intent_encoder"]),
            num_categories=len(self.label_encoders["category_encoder"]),
            num_ner_labels=len(self.label_encoders["ner_label_encoder"]),
            dropout_rate=self.hyperparameters["dropout_rate"]
        )
        
        if self.gpt2_tokenizer.pad_token_id is not None:
            self.classification_model.gpt2.resize_token_embeddings(len(self.gpt2_tokenizer))
        
        self.classification_model.load_state_dict(torch.load(model_path, map_location=device), strict=False)
        self.classification_model.to(device)
        self.classification_model.eval()
        
        # Load generation model using from_pretrained instead of torch.load
        try:
            print(f"Attempting to load generation model from {self.generation_model_path}")
            self.generation_model = GPT2LMHeadModel.from_pretrained(self.generation_model_path).to(device)
            self.generation_tokenizer = GPT2Tokenizer.from_pretrained(self.generation_tokenizer_path)
            self.generation_tokenizer.pad_token = self.generation_tokenizer.eos_token
            self.generation_tokenizer.add_special_tokens({'additional_special_tokens': ['[INST]', '[RESP]', '[EOS]']})
            self.generation_model.resize_token_embeddings(len(self.generation_tokenizer))
        except Exception as e:
            print(f"Error loading generation model: {e}")
            print("Falling back to default GPT2...")
            self.generation_model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
            self.generation_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
            self.generation_tokenizer.pad_token = self.generation_tokenizer.eos_token
            self.generation_tokenizer.add_special_tokens({'additional_special_tokens': ['[INST]', '[RESP]', '[EOS]']})
            self.generation_model.resize_token_embeddings(len(self.generation_tokenizer))
        
        self.generation_model.eval()
        
        return self.classification_model, self.generation_model, self.gpt2_tokenizer, self.distilbert_tokenizer, self.generation_tokenizer
    
    def run_benchmark(self):
        # Load test data
        with open(self.test_data_path, "r", encoding="utf-8") as f:
            test_data = json.load(f)
            
        # Select samples for testing
        if len(test_data) > self.num_samples:
            test_samples = test_data[:self.num_samples]
        else:
            test_samples = test_data
            
        print(f"Running performance test on {len(test_samples)} samples")
        
        # Run tests on each device
        for device in self.devices:
            print(f"\n{'='*80}")
            print(f"RUNNING BENCHMARK ON {device.upper()}")
            print(f"{'='*80}")
            
            classification_model, generation_model, gpt2_tokenizer, distilbert_tokenizer, generation_tokenizer = self.load_models(device)
            
            for idx, sample in enumerate(tqdm(test_samples, desc=f"Testing on {device.upper()}")):
                instruction = sample["instruction"]
                
                # Measure classification time and memory
                def run_classification():
                    return inference_hybrid(
                        classification_model, 
                        instruction, 
                        gpt2_tokenizer, 
                        distilbert_tokenizer, 
                        self.label_encoders, 
                        self.hyperparameters["max_length"],
                        device
                    )
                
                classification_start = time.time()
                classification_result, classification_memory = get_peak_memory_usage(
                    run_classification, device=device
                )
                classification_end = time.time()
                classification_time = classification_end - classification_start
                
                # Create new input with classification results
                intent = classification_result["intent"]["label"]
                category = classification_result["category"]["label"]
                entities_text = ", ".join([f"{entity['entity']} ({entity['label']})" for entity in classification_result["ner"]]) if classification_result["ner"] else "none"
                new_input = f"{instruction} [Classified: Intent is '{intent}', Category is '{category}', Entities are {entities_text}]"
                
                # Measure generation time and memory
                def run_generation():
                    return generate_response(
                        generation_model,
                        generation_tokenizer,
                        instruction,
                        classification_result,
                        device=device
                    )
                
                generation_start = time.time()
                generated_response, generation_memory = get_peak_memory_usage(
                    run_generation, device=device
                )
                generation_end = time.time()
                generation_time = generation_end - generation_start
                
                # Calculate overall time and memory
                overall_time = classification_time + generation_time
                overall_memory = classification_memory + generation_memory
                
                # Store results
                result = {
                    "sample_id": idx + 1,
                    "instruction": instruction,
                    "new_input": new_input,
                    "generated_response": generated_response,
                    "classification_time": classification_time,
                    "generation_time": generation_time,
                    "overall_time": overall_time,
                    "classification_memory": classification_memory,
                    "generation_memory": generation_memory,
                    "overall_memory": overall_memory
                }
                self.results[device].append(result)
                
                # Update summary statistics
                self.summary[device]["classification_time"].append(classification_time)
                self.summary[device]["generation_time"].append(generation_time)
                self.summary[device]["overall_time"].append(overall_time)
                self.summary[device]["memory_usage"].append(overall_memory)
                
                # Print sample result
                print(f"\nSample {idx+1}/{len(test_samples)} (Device: {device.upper()}):")
                print(f"Instruction: {instruction}")
                print(f"New Input: {new_input}")
                print(f"Generated Response: {generated_response}")
                print(f"Model                | Classification Time (s)   | Text Generation Time (s)  | Overall Inference Time (s) | Device     | Memory Usage (MB)")
                print(f"Hybrid Cross Attention | {classification_time:.4f}                    | {generation_time:.4f}                   | {overall_time:.4f}                   | {device.upper()}        | {overall_memory:.4f}")
            
            # Calculate average times
            self.summary[device]["avg_classification_time"] = sum(self.summary[device]["classification_time"]) / len(test_samples)
            self.summary[device]["avg_generation_time"] = sum(self.summary[device]["generation_time"]) / len(test_samples)
            self.summary[device]["avg_overall_time"] = sum(self.summary[device]["overall_time"]) / len(test_samples)
            self.summary[device]["avg_memory_usage"] = sum(self.summary[device]["memory_usage"]) / len(test_samples)
            
            print(f"\n{'='*80}")
            print(f"BENCHMARK SUMMARY FOR {device.upper()}")
            print(f"{'='*80}")
            print(f"Average Classification Time: {self.summary[device]['avg_classification_time']:.4f} seconds")
            print(f"Average Generation Time: {self.summary[device]['avg_generation_time']:.4f} seconds")
            print(f"Average Overall Time: {self.summary[device]['avg_overall_time']:.4f} seconds")
            print(f"Average Memory Usage: {self.summary[device]['avg_memory_usage']:.4f} MB")

        # Save results to files and create plots
        self.save_results()

    # Add to your PerformanceTest class
    def save_results(self):
        """Save benchmark results to JSON files in the specified format"""
        
        # Save CPU results
        if "cpu" in self.devices and "avg_classification_time" in self.summary["cpu"]:
            cpu_results = []
            
            for result in self.results["cpu"]:
                formatted_result = {
                    "Model": "Hybrid Cross attention",
                    "Classification Time (s)": result["classification_time"],
                    "Text Generation Time (s)": result["generation_time"],
                    "Overall Inference Time (s)": result["overall_time"],
                    "Device": "CPU",
                    "Memory Usage (MB)": result["overall_memory"],
                    "Instruction": result["instruction"],
                    "New Input": result["new_input"],
                    "Generated Response": result["generated_response"]
                }
                cpu_results.append(formatted_result)
            
            with open("performance_results_cpu.json", "w") as f:
                json.dump(cpu_results, f, indent=4)
            print("CPU results saved to performance_results_cpu.json")
        
        # Save GPU results
        if "cuda" in self.devices and "avg_classification_time" in self.summary["cuda"]:
            gpu_results = []
            
            for result in self.results["cuda"]:
                formatted_result = {
                    "Model": "Hybrid Cross Attention",
                    "Classification Time (s)": result["classification_time"],
                    "Text Generation Time (s)": result["generation_time"],
                    "Overall Inference Time (s)": result["overall_time"],
                    "Device": "GPU",
                    "Memory Usage (MB)": result["overall_memory"],
                    "Instruction": result["instruction"],
                    "New Input": result["new_input"],
                    "Generated Response": result["generated_response"]
                }
                gpu_results.append(formatted_result)
            
            with open("performance_results_gpu.json", "w") as f:
                json.dump(gpu_results, f, indent=4)
            print("GPU results saved to performance_results_gpu.json")
        
        # Create performance log and comparison plots
        self.create_performance_log()
        self.create_comparison_plots()

    def create_performance_log(self):
        """Create a text file with performance comparisons"""
        with open("performance_log.txt", "w") as f:
            f.write("=== PERFORMANCE BENCHMARK RESULTS ===\n\n")
            
            # Write CPU results
            if "cpu" in self.devices:
                f.write("CPU PERFORMANCE:\n")
                f.write(f"Average Classification Time: {self.summary['cpu']['avg_classification_time']:.4f} seconds\n")
                f.write(f"Average Generation Time: {self.summary['cpu']['avg_generation_time']:.4f} seconds\n")
                f.write(f"Average Overall Time: {self.summary['cpu']['avg_overall_time']:.4f} seconds\n")
                f.write(f"Average Memory Usage: {self.summary['cpu']['avg_memory_usage']:.4f} MB\n\n")
            
            # Write GPU results
            if "cuda" in self.devices and "cuda" in self.results:
                f.write("GPU PERFORMANCE:\n")
                f.write(f"Average Classification Time: {self.summary['cuda']['avg_classification_time']:.4f} seconds\n")
                f.write(f"Average Generation Time: {self.summary['cuda']['avg_generation_time']:.4f} seconds\n")
                f.write(f"Average Overall Time: {self.summary['cuda']['avg_overall_time']:.4f} seconds\n")
                f.write(f"Average Memory Usage: {self.summary['cuda']['avg_memory_usage']:.4f} MB\n\n")
            
            # Write comparison
            if "cpu" in self.devices and "cuda" in self.devices and "cuda" in self.results:
                f.write("CPU vs GPU COMPARISON:\n")
                class_speedup = self.summary['cpu']['avg_classification_time'] / self.summary['cuda']['avg_classification_time']
                gen_speedup = self.summary['cpu']['avg_generation_time'] / self.summary['cuda']['avg_generation_time']
                overall_speedup = self.summary['cpu']['avg_overall_time'] / self.summary['cuda']['avg_overall_time']
                
                f.write(f"Classification Speed Improvement: {class_speedup:.2f}x faster on GPU\n")
                f.write(f"Generation Speed Improvement: {gen_speedup:.2f}x faster on GPU\n")
                f.write(f"Overall Speed Improvement: {overall_speedup:.2f}x faster on GPU\n\n")
                
                f.write("Notes:\n")
                f.write("- GPU memory usage is typically higher but processing is faster\n")
                f.write("- The generation task shows the largest speed improvement on GPU\n")
            
            f.write("\n=== END OF REPORT ===\n")
        
        print("Performance log saved to performance_log.txt")

    def create_comparison_plots(self):
        """Create matplotlib comparison plots"""
        if "cpu" not in self.devices or "cuda" not in self.devices or "cuda" not in self.results:
            print("Both CPU and GPU results are needed for comparison plots")
            return
        
        # Set up the plots
        plt.figure(figsize=(16, 10))
        
        # 1. Processing Time Comparison
        plt.subplot(2, 2, 1)
        
        labels = ['Classification', 'Generation', 'Overall']
        cpu_times = [self.summary['cpu']['avg_classification_time'], 
                    self.summary['cpu']['avg_generation_time'],
                    self.summary['cpu']['avg_overall_time']]
        
        gpu_times = [self.summary['cuda']['avg_classification_time'], 
                    self.summary['cuda']['avg_generation_time'],
                    self.summary['cuda']['avg_overall_time']]
        
        x = np.arange(len(labels))
        width = 0.35
        
        plt.bar(x - width/2, cpu_times, width, label='CPU')
        plt.bar(x + width/2, gpu_times, width, label='GPU')
        
        plt.xlabel('Task')
        plt.ylabel('Time (seconds)')
        plt.title('Average Processing Time Comparison')
        plt.xticks(x, labels)
        plt.legend()
        
        # Add value labels on the bars
        for i, v in enumerate(cpu_times):
            plt.text(i - width/2, v + 0.01, f"{v:.2f}s", ha='center')
        
        for i, v in enumerate(gpu_times):
            plt.text(i + width/2, v + 0.01, f"{v:.2f}s", ha='center')
        
        # 2. Speed Improvement
        plt.subplot(2, 2, 2)
        
        speedups = [
            cpu_times[0] / gpu_times[0] if gpu_times[0] > 0 else 0,
            cpu_times[1] / gpu_times[1] if gpu_times[1] > 0 else 0,
            cpu_times[2] / gpu_times[2] if gpu_times[2] > 0 else 0
        ]
        
        plt.bar(x, speedups, width=0.5)
        plt.axhline(y=1.0, color='r', linestyle='-', alpha=0.3)
        plt.xlabel('Task')
        plt.ylabel('Times Faster on GPU')
        plt.title('CPU vs GPU Speed Improvement')
        plt.xticks(x, labels)
        
        # Add speed improvement labels
        for i, v in enumerate(speedups):
            plt.text(i, v + 0.1, f"{v:.1f}x", ha='center')
        
        # 3. Memory Usage
        plt.subplot(2, 2, 3)
        
        memory_labels = ['CPU', 'GPU']
        memory_usage = [self.summary['cpu']['avg_memory_usage'],
                    self.summary['cuda']['avg_memory_usage']]
        
        plt.bar(memory_labels, memory_usage)
        plt.xlabel('Device')
        plt.ylabel('Memory Usage (MB)')
        plt.title('Average Memory Usage')
        
        # Add memory usage labels
        for i, v in enumerate(memory_usage):
            plt.text(i, v + 0.1, f"{v:.1f} MB", ha='center')
        
        # 4. Time breakdown for both devices
        plt.subplot(2, 2, 4)
        
        # CPU breakdown
        cpu_breakdown = [
            self.summary['cpu']['avg_classification_time'] / self.summary['cpu']['avg_overall_time'] * 100,
            self.summary['cpu']['avg_generation_time'] / self.summary['cpu']['avg_overall_time'] * 100
        ]
        
        # GPU breakdown  
        gpu_breakdown = [
            self.summary['cuda']['avg_classification_time'] / self.summary['cuda']['avg_overall_time'] * 100,
            self.summary['cuda']['avg_generation_time'] / self.summary['cuda']['avg_overall_time'] * 100
        ]
        
        task_labels = ['Classification', 'Generation']
        
        x = np.arange(len(memory_labels))
        width = 0.35
        
        plt.bar(x - width/2, [cpu_breakdown[0], gpu_breakdown[0]], width, label='Classification')
        plt.bar(x - width/2, [cpu_breakdown[1], gpu_breakdown[1]], width, bottom=[cpu_breakdown[0], gpu_breakdown[0]], 
                label='Generation')
        
        plt.xlabel('Device')
        plt.ylabel('Percentage of Overall Time (%)')
        plt.title('Task Time Distribution')
        plt.xticks(x - width/2, memory_labels)
        plt.legend()
        
        # Add percentage labels
        for i, (c, g) in enumerate(zip(cpu_breakdown, gpu_breakdown)):
            plt.text(0 - width/2, c/2 if i == 0 else cpu_breakdown[0] + c/2, 
                    f"{c:.1f}%", ha='center')
            plt.text(1 - width/2, g/2 if i == 0 else gpu_breakdown[0] + g/2,
                    f"{g:.1f}%", ha='center')
        
        plt.tight_layout()
        plt.savefig('performance_comparison.png')
        plt.close()
        
        print("Performance comparison plots saved to performance_comparison.png")
def main():
    # Define paths
    model_paths = {
        "hybrid_model_dir": "../../Hybrid_Cross_Attention_Freeze",
        "generation_model_path": "../../text_generation_results_03-09-25/model",
        "generation_tokenizer_path": "../../text_generation_results_03-09-25/tokenizer"
    }
    
    test_data_path = "../../test.json"
    
    # Verify and print paths
    print(f"Hybrid model directory: {model_paths['hybrid_model_dir']}")
    print(f"Test data path: {test_data_path}")
    
    # Number of samples to test
    num_samples = 50  # You can adjust this number
    
    # Run the benchmark
    benchmark = PerformanceTest(model_paths, test_data_path, num_samples)
    benchmark.run_benchmark()

if __name__ == "__main__":
    # Check available devices
    device_info = "CUDA" if torch.cuda.is_available() else "CPU"
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    print(f"Running on: {device_info}")
    
    # Run the benchmark
    main()

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedba

GPU 0: NVIDIA GeForce RTX 3050 Laptop GPU
Running on: CUDA
Hybrid model directory: ../../Hybrid_Cross_Attention_Freeze
Test data path: ../../test.json
Running performance test on 50 samples

RUNNING BENCHMARK ON CPU

Loading models on CPU...
Loading from: ../../Hybrid_Cross_Attention_Freeze\label_encoders.json
Initializing model...
All GPT-2 and DistilBERT layers remain frozen


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Attempting to load generation model from ../../text_generation_results_03-09-25/model


Testing on CPU:   2%|▏         | 1/50 [00:17<14:04, 17.24s/it]

...

Testing on CUDA:  98%|█████████▊| 49/50 [02:12<00:02,  2.54s/it]


Sample 49/50 (Device: CUDA):
Instruction: change  order ord-8335461
New Input: change  order ord-8335461 [Classified: Intent is 'cancel_order', Category is 'order', Entities are order (order_number), ord-8335461 (order_number)]
Generated Response: i understand that you need assistance with changing order number {{order number}}. i'm here to help you with that. could you please provide me with the specific details of the changes you would like to make to your order? this will allow me to guide you through the process and ensure a seamless experience.
Model                | Classification Time (s)   | Text Generation Time (s)  | Overall Inference Time (s) | Device     | Memory Usage (MB)
Hybrid Cross Attention | 0.1580                    | 1.6634                   | 1.8215                   | CUDA        | 2700.4033


Testing on CUDA: 100%|██████████| 50/50 [02:14<00:00,  2.69s/it]


Sample 50/50 (Device: CUDA):
Instruction: i need assistance to send some feedback for ur produucts
New Input: i need assistance to send some feedback for ur produucts [Classified: Intent is 'review', Category is 'feedback', Entities are none]
Generated Response: we appreciate your willingness to share your feedback about our products and services. your insights play a crucial role in helping us improve and deliver a better experience to our valued customers. please let me know if you have any specific questions or if there's anything else i can assist you with.
Model                | Classification Time (s)   | Text Generation Time (s)  | Overall Inference Time (s) | Device     | Memory Usage (MB)
Hybrid Cross Attention | 0.1593                    | 1.8184                   | 1.9777                   | CUDA        | 2696.9136

BENCHMARK SUMMARY FOR CUDA
Average Classification Time: 0.2105 seconds
Average Generation Time: 2.4827 seconds
Average Overall Time: 2.6932 seconds
Average Memo

Performance comparison plots saved to performance_comparison.png
